In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("tr1gg3rtrash/yoga-posture-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/yoga-posture-dataset


In [ ]:
import os
from matplotlib import pyplot as plt
from PIL import Image

In [ ]:
# Set dataset path
dataset_path = "/kaggle/input/yoga-posture-dataset"

# List pose folders
poses = os.listdir(dataset_path)
print(f"Total poses: {len(poses)}")

In [ ]:
# Filter out non-directory items
poses = [pose for pose in poses if os.path.isdir(os.path.join(dataset_path, pose))]
print(f"Total valid pose folders: {len(poses)}")

In [ ]:
# Visualize sample images
for pose in poses[:5]:
    pose_path = os.path.join(dataset_path, pose)
    sample_image = os.listdir(pose_path)[0]
    image_path = os.path.join(pose_path, sample_image)

    img = Image.open(image_path)
    plt.imshow(img)
    plt.title(pose)
    plt.axis("off")
    plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
# Paths for train and test datasets
train_dir = "/kaggle/working/train"
test_dir = "/kaggle/working/test"

In [ ]:
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
print("Source Path:", os.path.join(pose_path, img))
print("Destination Path:", os.path.join(train_dir, pose, img))


In [ ]:
# Process each pose folder
for pose in poses:
    pose_path = os.path.join(dataset_path, pose)
    images = os.listdir(pose_path)
    
    # Split images into train and test sets
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)
    
    # Ensure subdirectories exist for each pose
    os.makedirs(os.path.join(train_dir, pose), exist_ok=True)
    os.makedirs(os.path.join(test_dir, pose), exist_ok=True)
    
    # Copy files to train and test folders
    for img in train_images:
        shutil.copy(os.path.join(pose_path, img), os.path.join(train_dir, pose, img))
    for img in test_images:
        shutil.copy(os.path.join(pose_path, img), os.path.join(test_dir, pose, img))

In [ ]:
print("Source Path:", os.path.join(pose_path, img))
print("Destination Path:", os.path.join(train_dir, pose, img))


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model

In [ ]:
# Image dimensions
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

In [ ]:
# Load EfficientNetB0 pre-trained on ImageNet
base_model = EfficientNetB0(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
base_model.trainable = False  # Freeze the base model

In [ ]:
# Add custom layers for yoga pose classification
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Reduce spatial dimensions
x = Dense(128, activation='relu')(x)  # Fully connected layer
output = Dense(len(poses), activation='softmax')(x)  # Final layer with softmax for classification


In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2
)

In [ ]:
# Simple rescaling for validation
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

In [ ]:
for x, y in train_generator:
    print(f"Batch shape: {x.shape}, {y.shape}")
    break


# Train the Model

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint, early_stopping]
)


In [ ]:
print(f"Training samples: {train_generator.samples}")
print(f"Validation samples: {test_generator.samples}")


In [ ]:
print(f"Steps per epoch: {steps_per_epoch}")
print(f"Validation steps: {validation_steps}")

# Fine-Tune the Model

In [ ]:
# Unfreeze the base model for fine-tuning
base_model.trainable = True

In [ ]:
# Recompile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=5,
    steps_per_epoch=len(train_generator),
    validation_steps=len(test_generator)
)

In [ ]:
model.save("efficientnet_yoga_pose_model.h5")


# retrial


In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
from PIL import Image


In [ ]:
# Paths for dataset
dataset_path = "/kaggle/input/yoga-posture-dataset"
train_dir = "train_data"
test_dir = "test_data"

In [ ]:
# Parameters
IMG_SIZE = (224, 224)  # EfficientNet input size
BATCH_SIZE = 32


# 1. Data Preparation

In [ ]:
# Create directories for train and test splits
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [ ]:
# Function to handle transparency in images
def convert_to_rgb(image_path):
    with Image.open(image_path) as img:
        if img.mode in ("RGBA", "P"):
            img = img.convert("RGB")
        return img


In [ ]:
# Split images into train and test folders
from sklearn.model_selection import train_test_split
import shutil

In [ ]:
poses = [folder for folder in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, folder))]
for pose in poses:
    pose_path = os.path.join(dataset_path, pose)
    images = os.listdir(pose_path)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    os.makedirs(os.path.join(train_dir, pose), exist_ok=True)
    os.makedirs(os.path.join(test_dir, pose), exist_ok=True)

    for img in train_images:
        img_path = os.path.join(pose_path, img)
        shutil.copy(img_path, os.path.join(train_dir, pose, img))
    for img in test_images:
        img_path = os.path.join(pose_path, img)
        shutil.copy(img_path, os.path.join(test_dir, pose, img))


# 2. Data Augmentation and Generators

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0/255, rotation_range=20, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", shuffle=True
)
test_generator = test_datagen.flow_from_directory(
    test_dir, target_size=IMG_SIZE, batch_size=BATCH_SIZE, class_mode="categorical", shuffle=False
)

In [ ]:
# Add custom layers
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.3),
    layers.Dense(len(poses), activation="softmax")
])

# 3. Model Creation

In [ ]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model

In [ ]:
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# 4. Callbacks

In [ ]:
checkpoint = ModelCheckpoint("best_model.keras", monitor="val_accuracy", save_best_only=True, mode="max")

early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)


# 5. Training the Model

In [ ]:
steps_per_epoch = train_generator.samples // BATCH_SIZE
validation_steps = test_generator.samples // BATCH_SIZE

In [ ]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint, early_stopping]
)

# 6. Visualize Training Results

In [ ]:
def plot_training_history(history):
    plt.figure(figsize=(12, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(history.history["accuracy"], label="Train Accuracy")
    plt.plot(history.history["val_accuracy"], label="Validation Accuracy")
    plt.title("Accuracy Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy")
    plt.legend()

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(history.history["loss"], label="Train Loss")
    plt.plot(history.history["val_loss"], label="Validation Loss")
    plt.title("Loss Over Epochs")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.legend()

    plt.show()

In [ ]:
plot_training_history(history)

# 7. Evaluation

In [ ]:
# Generate predictions
y_true = test_generator.classes
y_pred = np.argmax(model.predict(test_generator), axis=1)


In [ ]:
# Confusion Matrix
conf_matrix = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=poses, yticklabels=poses)
plt.title("Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
# Classification Report
print(classification_report(y_true, y_pred, target_names=poses))

# third trial


In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2
from PIL import Image, ImageChops, ImageEnhance

In [ ]:
# Helper functions (for visualization and ELA)
def compute_ela_cv(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    SCALE = 15
    orig_img = cv2.imread(path)
    orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
    
    cv2.imwrite(temp_filename, orig_img, [cv2.IMWRITE_JPEG_QUALITY, quality])

    compressed_img = cv2.imread(temp_filename)
    diff = SCALE * cv2.absdiff(orig_img, compressed_img)
    return diff

In [ ]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpeg'
    ela_filename = 'temp_ela.png'
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality=quality)
    temp_image = Image.open(temp_filename)

    ela_image = ImageChops.difference(image, temp_image)

    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1

    scale = 255.0 / max_diff
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    return ela_image

In [ ]:
def random_sample(path, extension=None):
    if extension:
        items = Path(path).glob(f'*.{extension}')
    else:
        items = Path(path).glob(f'*')
        
    items = list(items)
    p = np.random.choice(items)
    return p.as_posix()

In [ ]:
# Set batch size and image size
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)

In [ ]:
# Load dataset
dataset = "../input/yoga-posture-dataset"
image_dir = Path(dataset)
filepaths = list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))


In [ ]:
# Create DataFrame
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1)

In [ ]:
# Display a random sample of images
random_index = np.random.randint(0, len(image_df), 16)
fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(10, 10), subplot_kw={'xticks': [], 'yticks': []})
for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(image_df.Filepath[random_index[i]]))
    ax.set_title(image_df.Label[random_index[i]])
plt.tight_layout()
plt.show()

In [ ]:
# Split into train and test sets
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=1)


In [ ]:
# Data generators
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)


In [ ]:
test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)


In [ ]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)

In [ ]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='validation'
)

In [ ]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

In [ ]:
# Load EfficientNetB0 model
base_model = EfficientNetB0(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet',
    pooling='avg'
)

In [ ]:
base_model.trainable = False

In [ ]:
# Define model architecture
inputs = tf.keras.Input(shape=(224, 224, 3))
x = base_model(inputs)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
outputs = Dense(len(train_images.class_indices), activation='softmax')(x)
model = Model(inputs, outputs)

In [ ]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Callbacks
checkpoint = ModelCheckpoint("best_model.keras", save_best_only=True, monitor="val_accuracy", mode="max")
early_stopping = EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
tensorboard = TensorBoard(log_dir="logs")

In [ ]:
def preprocess_image(image):
    if image.mode != "RGB":
        image = image.convert("RGB")
    return image


In [ ]:
# Check the number of samples
print(f"Training samples: {train_images.samples}")
print(f"Validation samples: {val_images.samples}")


In [ ]:
# Ensure correct steps per epoch
steps_per_epoch = train_images.samples // train_images.batch_size
validation_steps = val_images.samples // val_images.batch_size

# Train the model
history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=10,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps,
    callbacks=[checkpoint, early_stopping, tensorboard],
    verbose=1  # Display training progress
)


In [ ]:
model.summary()


In [ ]:
print("Class indices:", train_images.class_indices)
print("Number of classes:", len(train_images.class_indices))


In [ ]:
# Evaluate on test data
results = model.evaluate(test_images, verbose=1)

# Display the results
print(f"Test Loss: {results[0]:.5f}")
print(f"Test Accuracy: {results[1] * 100:.2f}%")


In [ ]:
# Generate predictions
predictions = model.predict(test_images)

# Convert predictions to class labels
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_images.classes


In [ ]:
from sklearn.metrics import classification_report

# Classification report
class_names = list(test_images.class_indices.keys())  # Retrieve class names
report = classification_report(true_classes, predicted_classes, target_names=class_names)
print(report)


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
def plot_history(history):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Val Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Val Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()



In [ ]:

# Call the function to plot
plot_history(history)

In [ ]:
# Get misclassified indices
misclassified_indices = np.where(predicted_classes != true_classes)[0]

In [ ]:
# Display a few misclassified examples
import random
random_indices = random.sample(list(misclassified_indices), 9)
plt.figure(figsize=(10, 10))


In [ ]:
for i, idx in enumerate(random_indices):
    plt.subplot(3, 3, i + 1)
    img = plt.imread(test_images.filepaths[idx])
    plt.imshow(img)
    plt.title(f"True: {class_names[true_classes[idx]]}, Pred: {class_names[predicted_classes[idx]]}")
    plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
print(f"Training samples: {train_images.samples}")
print(f"Validation samples: {val_images.samples}")


# another trial

In [24]:
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
from pathlib import Path
import cv2
from PIL import Image, ImageChops, ImageEnhance

In [3]:
# === Helper Functions ===
def preprocess_image(image):
    if image.mode != "RGB":
        image = image.convert("RGB")
    return image

In [25]:
# Hyperparameters and configurations
NUM_CLASSES = 43  # Replace with your actual number of classes
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4

In [26]:
dataset = "../input/yoga-posture-dataset"  # Replace with your dataset path
image_dir = Path(dataset)

In [27]:
filepaths = list(image_dir.glob(r'**/*.jpg')) + list(image_dir.glob(r'**/*.JPG')) + list(image_dir.glob(r'**/*.png'))
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))


In [28]:
# Create DataFrame
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1)

In [29]:
# Split into train and test sets
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=1)


In [30]:
# Data Generators
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

In [31]:
test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [32]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)

Found 1508 validated image filenames belonging to 43 classes.


In [33]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='validation'
)

Found 376 validated image filenames belonging to 43 classes.


In [13]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 471 validated image filenames belonging to 40 classes.


In [34]:
# Identify and handle missing classes in the test set
num_classes = len(train_images.class_indices)  # Total number of classes in the dataset
missing_classes = set(range(num_classes)) - set(test_images.class_indices.values())
print(f"Missing classes in test set: {missing_classes}")

Missing classes in test set: {40, 41, 42}


In [39]:
# Adjust test generator labels dynamically
def adjust_test_labels(generator, num_classes):
    for x_batch, y_batch in generator:
        y_batch_one_hot = tf.one_hot(y_batch, depth=num_classes)
        yield x_batch, y_batch_one_hot

In [40]:
# === Model and Optimizer Comparison ===
models_to_test = {
    "EfficientNetB0": EfficientNetB0,
    "ResNet50": ResNet50,
}

In [41]:
optimizers_to_test = {
    "Adam": Adam(learning_rate=LEARNING_RATE),
    "SGD": SGD(learning_rate=LEARNING_RATE),
    "RMSprop": RMSprop(learning_rate=LEARNING_RATE),
}

In [42]:
# Data Generators
def prepare_data_generators(train_df, test_df):
    train_gen = tf.data.Dataset.from_tensor_slices((train_df['Filepath'], train_df['Label']))
    test_gen = tf.data.Dataset.from_tensor_slices((test_df['Filepath'], test_df['Label']))

    def preprocess(filepath, label):
        img = tf.io.read_file(filepath)
        img = tf.image.decode_image(img, channels=3)
        img = tf.image.resize(img, IMAGE_SIZE)
        img = tf.keras.applications.efficientnet.preprocess_input(img)
        label = tf.one_hot(label, depth=NUM_CLASSES)
        return img, label

    train_gen = train_gen.map(preprocess).batch(BATCH_SIZE).shuffle(1024)
    test_gen = test_gen.map(preprocess).batch(BATCH_SIZE)

    return train_gen, test_gen

results = {}

# Manually encode missing classes in the test set
from tensorflow.keras.utils import to_categorical

# Check if any class in test set is missing
missing_classes = set(range(43)) - set(test_images.class_indices.values())
print(f"Missing classes in test set: {missing_classes}")

# Ensure labels are encoded to fit the model's expected 43 output classes
for x_batch, y_batch in test_images:
    y_batch_one_hot = to_categorical(y_batch, num_classes=43)
    break  # Stop after checking the first batch

# Continue with evaluation
test_loss, test_accuracy = model.evaluate(test_images, verbose=0)


In [43]:
# Model Training
def train_model(base_model_fn, optimizer, train_gen, val_gen, steps_per_epoch, validation_steps):
    # Define the model
    base_model = base_model_fn(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))
    base_model.trainable = False  # Freeze the base model

    # Build the custom model
    inputs = tf.keras.Input(shape=(224, 224, 3))
    x = base_model(inputs)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    # Training loop
    loss_fn = tf.keras.losses.CategoricalCrossentropy()
    train_accuracy_metric = tf.keras.metrics.CategoricalAccuracy()
    val_accuracy_metric = tf.keras.metrics.CategoricalAccuracy()

    for epoch in range(EPOCHS):
        print(f"Epoch {epoch + 1}/{EPOCHS}")

        # Training step
        for step, (x_batch, y_batch) in enumerate(train_gen.take(steps_per_epoch)):
            with tf.GradientTape() as tape:
                logits = model(x_batch, training=True)
                loss = loss_fn(y_batch, logits)
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            train_accuracy_metric.update_state(y_batch, logits)

        # Validation step
        for x_batch, y_batch in val_gen.take(validation_steps):
            val_logits = model(x_batch, training=False)
            val_accuracy_metric.update_state(y_batch, val_logits)

        train_acc = train_accuracy_metric.result().numpy()
        val_acc = val_accuracy_metric.result().numpy()
        print(f"Train Accuracy: {train_acc:.4f}, Validation Accuracy: {val_acc:.4f}")

        # Reset metrics at the end of the epoch
        train_accuracy_metric.reset_states()
        val_accuracy_metric.reset_states()

    return model

In [44]:
# Evaluation Function
def evaluate_model(model, test_gen, test_steps):
    accuracy_metric = tf.keras.metrics.CategoricalAccuracy()
    for x_batch, y_batch in test_gen.take(test_steps):
        predictions = model(x_batch, training=False)
        accuracy_metric.update_state(y_batch, predictions)
    return accuracy_metric.result().numpy()

In [46]:
# === Execution ===
# Prepare train and test generators
train_gen, test_gen = prepare_data_generators(train_df, test_df)


ValueError: in user code:

    File "/tmp/ipykernel_23/1778104186.py", line 9, in preprocess  *
        img = tf.image.resize(img, IMAGE_SIZE)

    ValueError: 'images' contains no shape.


In [47]:
print(train_df.head())
print(test_df.head())


                                              Filepath               Label
983  ../input/yoga-posture-dataset/Vasisthasana/Fil...        Vasisthasana
916  ../input/yoga-posture-dataset/Vasisthasana/Fil...        Vasisthasana
700  ../input/yoga-posture-dataset/Urdhva Dhanurasa...  Urdhva Dhanurasana
572  ../input/yoga-posture-dataset/Ustrasana/File2.png           Ustrasana
553  ../input/yoga-posture-dataset/Ustrasana/File62...           Ustrasana
                                               Filepath  \
1685  ../input/yoga-posture-dataset/Setu Bandha Sarv...   
65    ../input/yoga-posture-dataset/Camatkarasana/Fi...   
1864  ../input/yoga-posture-dataset/Adho Mukha Svana...   
484   ../input/yoga-posture-dataset/Vrksasana/File53...   
576   ../input/yoga-posture-dataset/Ustrasana/File35...   

                         Label  
1685  Setu Bandha Sarvangasana  
65               Camatkarasana  
1864      Adho Mukha Svanasana  
484                  Vrksasana  
576                  Ustrasana

In [48]:
import os

print(all(os.path.exists(filepath) for filepath in train_df['Filepath']))
print(all(os.path.exists(filepath) for filepath in test_df['Filepath']))


True
True


In [49]:
for filepath in train_df['Filepath'].iloc[:5]:
    try:
        img = tf.io.read_file(filepath)
        img = tf.image.decode_image(img, channels=3)
        print(f"Successfully loaded: {filepath}")
    except Exception as e:
        print(f"Error loading {filepath}: {e}")


Successfully loaded: ../input/yoga-posture-dataset/Vasisthasana/File54.png
Successfully loaded: ../input/yoga-posture-dataset/Vasisthasana/File24.png
Successfully loaded: ../input/yoga-posture-dataset/Urdhva Dhanurasana/File42.png
Successfully loaded: ../input/yoga-posture-dataset/Ustrasana/File2.png
Successfully loaded: ../input/yoga-posture-dataset/Ustrasana/File62.png


In [ ]:
steps_per_epoch = math.ceil(len(train_df) / BATCH_SIZE)
validation_steps = math.ceil(len(test_df) / BATCH_SIZE)

In [ ]:
# Train and Evaluate Models
results = {}
for model_name, base_model_fn in models_to_test.items():
    for optimizer_name, optimizer in optimizers_to_test.items():
        print(f"Training {model_name} with {optimizer_name} optimizer...")

        # Train the model
        trained_model = train_model(base_model_fn, optimizer, train_gen, train_gen, steps_per_epoch, validation_steps)

        # Evaluate the model
        test_accuracy = evaluate_model(trained_model, test_gen, len(test_gen))
        print(f"Test Accuracy for {model_name} with {optimizer_name}: {test_accuracy:.4f}")
        results[(model_name, optimizer_name)] = test_accuracy

In [ ]:
# Display final results
print("\nFinal Results:")
for key, value in results.items():
    print(f"{key}: Accuracy = {value['accuracy']:.4f}, Loss = {value['loss']:.4f}")

In [ ]:
print(f"Class indices: {train_images.class_indices}")
print(f"Number of classes: {len(train_images.class_indices)}")


In [ ]:
print(f"Model output shape: {model.output_shape}")


In [ ]:
# Check batch shapes from test generator
for x_batch, y_batch in test_images:
    print(f"Input shape: {x_batch.shape}, Label shape: {y_batch.shape}")
    break

In [ ]:
print(f"Training set unique labels: {len(train_images.class_indices)}")



In [ ]:

print(f"Test set unique labels: {len(test_images.class_indices)}")


In [ ]:
# === Results Visualization ===
result_df = pd.DataFrame(results).T.reset_index()
result_df.columns = ["Model", "Optimizer", "Accuracy", "Loss"]

In [ ]:
# Plot Accuracy
plt.figure(figsize=(15, 6))
result_df.sort_values("Accuracy", ascending=False).plot(
    kind="bar", x="Model", y="Accuracy", color="skyblue", legend=False
)
plt.title("Model and Optimizer Accuracy Comparison")
plt.xlabel("Model + Optimizer")
plt.ylabel("Accuracy")
plt.xticks(rotation=45, ha="right")
plt.show()


In [ ]:
# Plot Loss
plt.figure(figsize=(15, 6))
result_df.sort_values("Loss").plot(
    kind="bar", x="Model", y="Loss", color="salmon", legend=False
)
plt.title("Model and Optimizer Loss Comparison")
plt.xlabel("Model + Optimizer")
plt.ylabel("Loss")
plt.xticks(rotation=45, ha="right")
plt.show()

In [ ]:
# Display sorted results
print("Model Comparison Results:")
print(result_df.sort_values("Accuracy", ascending=False))

# another trial another

In [50]:
import os
import math
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from sklearn.model_selection import train_test_split
from pathlib import Path

In [51]:
# === Helper Functions ===
def preprocess_image(image):
    if image.mode != "RGB":
        image = image.convert("RGB")
    return image


In [52]:
# === Configuration ===
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 1e-4
DATASET_PATH = "../input/yoga-posture-dataset"  # Replace with your dataset path

In [53]:
# === Load and Prepare Dataset ===
image_dir = Path(DATASET_PATH)
filepaths = list(image_dir.glob(r'**/*.jpg')) + \
            list(image_dir.glob(r'**/*.JPG')) + \
            list(image_dir.glob(r'**/*.png'))

In [54]:
labels = list(map(lambda x: os.path.split(os.path.split(x)[0])[1], filepaths))


In [55]:
filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')
image_df = pd.concat([filepaths, labels], axis=1)

In [56]:
# Split data
train_df, test_df = train_test_split(image_df, test_size=0.2, shuffle=True, random_state=1)


In [57]:
# Infer the number of classes
unique_classes = sorted(train_df['Label'].unique())
NUM_CLASSES = len(unique_classes)


In [58]:
# === Data Generators ===
train_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input,
    validation_split=0.2
)

In [59]:
test_generator = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.efficientnet.preprocess_input
)

In [60]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='training'
)

Found 1508 validated image filenames belonging to 43 classes.


In [61]:
val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=True,
    seed=42,
    subset='validation'
)

Found 376 validated image filenames belonging to 43 classes.


In [62]:
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Label',
    target_size=IMAGE_SIZE,
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    shuffle=False
)

Found 471 validated image filenames belonging to 40 classes.


In [63]:
# Handle missing classes in the test set
missing_classes = set(range(NUM_CLASSES)) - set(test_images.class_indices.values())
if missing_classes:
    print(f"Missing classes in test set: {missing_classes}")

Missing classes in test set: {40, 41, 42}


In [64]:
def adjust_test_labels(generator, num_classes):
    for x_batch, y_batch in generator:
        adjusted_labels = tf.keras.utils.to_categorical(
            np.array([generator.class_indices[label] for label in y_batch]),
            num_classes=num_classes
        )
        yield x_batch, adjusted_labels

In [65]:
# Adjust test labels
adjusted_test_gen = tf.data.Dataset.from_generator(
    lambda: adjust_test_labels(test_images, NUM_CLASSES),
    output_signature=(
        tf.TensorSpec(shape=(None, *IMAGE_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, NUM_CLASSES), dtype=tf.float32)
    )
)

In [66]:
# === Model Training ===
def train_model(base_model_fn, optimizer, train_gen, val_gen, num_classes):
    # Load base model
    base_model = base_model_fn(weights='imagenet', include_top=False, pooling='avg', input_shape=(*IMAGE_SIZE, 3))
    base_model.trainable = False

    # Add classification head
    inputs = tf.keras.Input(shape=(*IMAGE_SIZE, 3))
    x = base_model(inputs)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs, outputs)

    # Compile model
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
# === Training Loop ===
results = {}
for model_name, base_model_fn in [("EfficientNetB0", EfficientNetB0), ("ResNet50", ResNet50)]:
    for optimizer_name, optimizer in [("Adam", Adam(learning_rate=LEARNING_RATE)),
                                      ("SGD", SGD(learning_rate=LEARNING_RATE)),
                                      ("RMSprop", RMSprop(learning_rate=LEARNING_RATE))]:
        print(f"\nTraining {model_name} with {optimizer_name} optimizer...")
        model = train_model(base_model_fn, optimizer, train_images, val_images, NUM_CLASSES)

        # Train model
        history = model.fit(
            train_images,
            validation_data=val_images,
            epochs=EPOCHS,
            steps_per_epoch=len(train_images),
            validation_steps=len(val_images)
        )

        # Evaluate model
        test_accuracy = model.evaluate(adjusted_test_gen, steps=len(test_images))[1]
        print(f"Test Accuracy for {model_name} with {optimizer_name}: {test_accuracy:.4f}")
        results[(model_name, optimizer_name)] = test_accuracy


Training EfficientNetB0 with Adam optimizer...
Epoch 1/50


/opt/conda/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
